In [3]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
import numpy as np
from scipy.interpolate import griddata
from pyspark.sql.functions import col, struct, spark_partition_id
import math
import itertools
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id
from itertools import chain

spark = SparkSession.builder.getOrCreate()

#read the csv file and fix the headers
dataset = spark.read.csv("C:/Users/nikos/Desktop/partitiondata.csv", header=False).toDF("Id", "Text", "Latitude", "Longitude")

#retain the latitude, longitude, replace the characters and convert to float type 
latitude = dataset.rdd.map(lambda x: float(x[2].replace('"(', ''))).collect()
longitude = dataset.rdd.map(lambda x: float(x[3].replace(')"', ''))).collect()

#make the coordinates of x and y axis, from the minimum to the maximum value, and split each axis to 11 values
#thus we have a 10x10 grid
x, y = np.mgrid[min(longitude):max(longitude):11j, min(latitude):max(latitude):11j]

#the id of each cell of the grid
ids = np.zeros(len(longitude))

#chech each point of the dataset
for k in range(0,len(longitude)): 
    
    #check where longitude is less or equal to x, for each x of the grid 
    for idx, i in enumerate(x[1:,0]):
        if longitude[k] <= i:
            
            #check where latitude is less or equal to y, for each y of the grid 
            for jdx, j in enumerate(y[0,1:]):
                if latitude[k] <= j:
                    
                    #compute the id of the independent cell
                    ids[k] = idx*(len(x)-1) + jdx
                   
                    break
            break

#create a pandas dataframe with the ids of the cell and an increasing number to each row, as column
gridPandas = pd.DataFrame(ids, columns=['gridID'])
gridPandas.insert(0, 'IncreasingId', range(0, 0 + len(gridPandas)))

#create a spark dataframe from the pandas dataframe
gridPartition = spark.createDataFrame(gridPandas)

#create a column to the dataframe "dataset" with an increasing number in each row
data = dataset.withColumn("IncreasingId", monotonically_increasing_id())

#name the two dataframes
sqlContext.registerDataFrameAsTable(gridPartition, 'gridPartition')
sqlContext.registerDataFrameAsTable(data, 'data')

#join two dataframes based on the increasing number that has created
df2 = sqlContext.sql("SELECT data.Id, data.Text, data.Latitude, data.Longitude, gridPartition.gridID FROM data INNER JOIN gridPartition ON data.IncreasingId=gridPartition.IncreasingId")

#retain the distinct values of the cells which correspond the points
#from the 10x10 grid, only 17 cells have points
mapping = {k: i for i, k in enumerate(
    df2.select("gridID").distinct().rdd.flatMap(lambda x: x).collect()
)}

#partition by the distinct cell id of the grid
result = (df2
    .select("gridID", struct([c for c in df2.columns]))
    .rdd.partitionBy(len(mapping), lambda k: mapping[k])
    .values()
    .toDF(df2.schema))

print("Number of partitions: {}".format(result.rdd.getNumPartitions()))

partitions = result.rdd.glom().collect()
for i, l in enumerate(partitions): 
    print ("partition #{} length: {}".format(i, len(l)))

#result.write.partitionBy('gridID').format("csv").save('C:/Users/nikos/Desktop/gridPartitions')

#function which computes the jaccard similarity
def jaccard_similarity(str1, str2): 
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))

#function which computes the euclidean distance
def euclidean_distance(object1, object2):
    x1 = float(object1[2].replace('"(', ''))
    x2 = float(object2[2].replace('"(', ''))
    y1 = float(object1[3].replace(')"', ''))
    y2 = float(object2[3].replace(')"', ''))
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

#-------------------------------------------------- First function ---------------------------------------------------------

#first function which computes for each point against all, the distance and the textual similarity
#retain these that have distance less or equal to theta, and textual similarity greater than e
def func1(iterator):
    
    y = []
    
    for idx, obj in enumerate(iterator):
        x = []
        for objec in iterator:
            theta = 0.01
            e = 0.7
            
            if (objec!=obj) and (euclidean_distance(obj, objec) < theta) and (jaccard_similarity(obj[1], objec[1]) > e):
                x.append(objec[0])
                                 
        y.append(x)
    
    return y

#apply the first function to each partition
nearest_points_1 = result.rdd \
        .mapPartitions(func1) \
        .collect()
    
print("Nearest points for each partition from first function: {}".format(nearest_points_1))


#-------------------------------------------------- Second function ---------------------------------------------------------

#second function that sorts the points based on the x axis, and compute the distance for each point from its next points
#until the distance is greater than theta, which stops the computation for this point against the others
def func2(iterator):
    
    #retain the values of each partition to a pandas dataframe and sort them based on the longitude
    d = pd.DataFrame([p[0], p[1], float(p[2].replace('"(', '')), float(p[3].replace(')"', ''))] for p in iterator)
    d.columns = ['id', 'text', 'latitude', 'longitude']
    d.sort_values(by=['longitude'])
    
    #create a list of lists which saves the nearest points of each point
    points = [[] for x in range(len(d.index))]
    
    #check each point from its next points, until the distance is greater than theta
    for i, row1 in enumerate(d.iterrows()):
        for j, row2 in enumerate(d[i+1:].iterrows()):
            theta = 0.001
            e = 0.7
            
            if math.sqrt((row1[1][2] - row2[1][2])**2 + (row1[1][3] - row2[1][3])**2) < theta:
                    if jaccard_similarity(row1[1][1], row2[1][1]) > e:
                        points[i].append(row2[1][0])
                        points[j].append(row1[1][0])
            else:
                break
                                         
    return points

#apply the second function to each partition
nearest_points_2 = result.rdd \
        .mapPartitions(func2) \
        .collect()
    
print("Nearest points for each partition from second function: {}".format(nearest_points_2))

Number of partitions: 17
partition #0 length: 1659
partition #1 length: 1
partition #2 length: 72
partition #3 length: 52
partition #4 length: 69
partition #5 length: 2176
partition #6 length: 130
partition #7 length: 656
partition #8 length: 1622
partition #9 length: 1245
partition #10 length: 411
partition #11 length: 19
partition #12 length: 1157
partition #13 length: 195
partition #14 length: 462
partition #15 length: 73
partition #16 length: 1
Nearest points for each partition from first function: [[' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005', ' 1005'], [], [' 1005', ' 1005'